As you will see, the microarray 'processed' data obtained from the array express dataset is not really processed.

**data ref:** https://www.pnas.org/content/105/16/6156

In [6]:
import pandas as pd
import re

In [4]:
data_file_path = 'ArrayExpressData/E-TABM-337/'
processed_matrix = 'FinalDataMatrix.txt'

stockel_raw = pd.read_csv(data_file_path+processed_matrix,sep='\t',header=[0,1])
stockel_raw.head()

Scan REF  \
                Reporter REF   
0  Contig0.10_10_14370_15446   
1  Contig0.10_11_16336_15422   
2  Contig0.10_12_16503_17552   
3  Contig0.10_13_17679_17981   
4  Contig0.10_14_18000_18698   

  BioRep1_Day1_L9_Cy3_1;BioRep1_Day1_L9_Cy3_2;BioRep1_Day1_L9_Cy5_1;BioRep2_Day1_L9_Cy3_1;BioRep2_Day1_L9_Cy3_2;BioRep2_Day1_L9_Cy5_1  \
                                                                               Software Unknown:Replicates Mean Normalized Log2 Ratio   
0                                              -0.16                                                                                    
1                                               0.23                                                                                    
2                                              -0.04                                                                                    
3                                              -0.08                                                                                    
4                                              -0.15                                                                                    

                                                                 \
  Software Unknown:Replicates SD Software Unknown:Replicates SE   
0                           0.05                           0.02   
1                           0.05                           0.01   
2                           0.12                           0.03   
3                           0.16                           0.04   
4                           0.14                           0.04   

  BioRep1_Day1_D9_Cy3_1;BioRep1_Day1_D9_Cy3_2;BioRep1_Day1_D9_Cy5_1;BioRep2_Day1_D9_Cy3_1;BioRep2_Day1_D9_Cy3_2;BioRep2_Day1_D9_Cy5_1  \
                                                                               Software Unknown:Replicates Mean Normalized Log2 Ratio   
0                                              -0.13                                                                                    
1                                              -0.31                                                                                    
2                                              -0.41                                                                                    
3                                               0.13                                                                                    
4                                              -0.02                                                                                    

                                                                 \
  Software Unknown:Replicates SD Software Unknown:Replicates SE   
0                           0.04                           0.01   
1                           0.09                           0.02   
2                           0.08                           0.02   
3                           0.09                           0.03   
4                           0.12                           0.03   

  BioRep1_Day1_D1_Cy3_1;BioRep1_Day1_D1_Cy3_2;BioRep1_Day1_D1_Cy5_1;BioRep1_Day1_D1_Cy5_2;BioRep2_Day1_D1_Cy3_1;BioRep2_Day1_D1_Cy3_2;BioRep2_Day1_D1_Cy5_1;BioRep2_Day1_D1_Cy5_2  \
                                                                                                                           Software Unknown:Replicates Mean Normalized Log2 Ratio   
0                                              -0.74                                                                                                                                
1                                              -0.33                                                                                                                                
2                                              -0.46                                                                                                                                
3                             

The columns are all messed up (starts from Day1 L9), index has contigs (we'll get proper gene names), the rest of the data looks good (it is already normalized and log2 values are calculated, sd and se are also given). 

We will arrange the columns referring the paper (link given above). The order will be Day1_D1, Day1_D5, Day1_D9, Day1_L1, Day1_L5, Day1_L9, Day2_D1, Day2_D5, Day2_D9, Day2_L1, Day2_L5, Day2_L9.

The column has 2 levels, we'll edit the column names in both the levels and follow the above pattern.  

In [7]:
def colNameFunc(name):
    names = name.split(';')
    pattern = re.compile(r'^BioRep[1|2]_(Day[1|2])_([D|L][1-9])_Cy[3|5]_[1|2]$')
    matched = re.match(pattern,names[0])
    
    return '_'.join([matched.group(1),matched.group(2)]) if matched else names[0]

def colNameLevel1(name):
    names = name.split(':')
    if len(names)==1:
        return names[0]
    replace = {'Replicates Mean Normalized Log2 Ratio': 'Exp-Val','Replicates SD': 'SD', 'Replicates SE':'SE'}
    return replace[names[1]]

stockel_new_col_names = list(map(colNameFunc,list(stockel_raw.columns.levels[0])))
stockel_new_col_level1 = list(map(colNameLevel1,list(stockel_raw.columns.levels[1])))

In [8]:
stockel_raw.columns = stockel_raw.columns.set_levels(stockel_new_col_names,level=0)
stockel_raw.columns = stockel_raw.columns.set_levels(stockel_new_col_level1,level=1)

Next we will only keep the log2 ratio column and the contigs, and merge into a single column level.

In [37]:
df = stockel_raw.loc[:,stockel_raw.columns.get_level_values(1).isin({'Exp-Val','Reporter REF'})]

Then we will sort the column order as given above.

In [38]:
df.columns = df.columns.droplevel(1)

In [41]:
df = df.reindex(sorted(df.columns),axis=1).set_index('Scan REF')

In [42]:
df.head()

,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9
Scan REF,,,,,,,,,,,,
Contig0.10_10_14370_15446,-0.74,-0.62,-0.13,0.35,0.26,-0.16,-0.87,-0.43,-0.12,0.38,0.37,-0.04
Contig0.10_11_16336_15422,-0.33,-0.64,-0.31,0.16,0.41,0.23,-0.42,-0.45,-0.30,0.06,0.34,0.23
Contig0.10_12_16503_17552,-0.46,-0.41,-0.41,0.66,-0.20,-0.04,-0.46,-0.28,-0.46,0.48,-0.01,0.04
Contig0.10_13_17679_17981,-0.23,0.27,0.13,0.08,-0.12,-0.08,-0.23,0.11,0.12,0.18,-0.05,-0.08
Contig0.10_14_18000_18698,-0.18,0.18,-0.02,-0.09,-0.22,-0.15,-0.21,0.09,-0.04,-0.08,-0.13,-0.16


Next we will map the contigs to the gene names. For that, we will need the contig to gene mappings.

In [48]:
mappings = 'Contig-ORF.txt'
stockel_contig = pd.read_csv(data_file_path+mappings,sep='\t',header=None)
stockel_contig.columns = ['Contig','ORF']

In [50]:
stockel_contig.head()

,Contig,ORF
0,ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-864.Cont...,cce_4187
1,ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-864.Cont...,cce_4186
2,ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-864.Cont...,cce_4185
3,ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-864.Cont...,cce_4184
4,ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-864.Cont...,cce_4183


Need to parse the contig columns properly and apply the function to all the rows of that columns. 

In [51]:
def ContigFunc(name):
    return name.replace('ebi.ac.uk:MIAMExpress:Reporter:A-MEXP-864.','')

stockel_contig.Contig = stockel_contig.Contig.apply(ContigFunc)
stockel_contig = stockel_contig.iloc[:-1].set_index('Contig')

Creating a new dataframe by merging information from both the dataframes

In [52]:
new_df = stockel_contig.join(df)

In [53]:
new_df.head()

,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9
Contig,,,,,,,,,,,,,
Contig0.10_10_14370_15446,cce_4187,-0.74,-0.62,-0.13,0.35,0.26,-0.16,-0.87,-0.43,-0.12,0.38,0.37,-0.04
Contig0.10_11_16336_15422,cce_4186,-0.33,-0.64,-0.31,0.16,0.41,0.23,-0.42,-0.45,-0.30,0.06,0.34,0.23
Contig0.10_12_16503_17552,cce_4185,-0.46,-0.41,-0.41,0.66,-0.20,-0.04,-0.46,-0.28,-0.46,0.48,-0.01,0.04
Contig0.10_13_17679_17981,cce_4184,-0.23,0.27,0.13,0.08,-0.12,-0.08,-0.23,0.11,0.12,0.18,-0.05,-0.08
Contig0.10_14_18000_18698,cce_4183,-0.18,0.18,-0.02,-0.09,-0.22,-0.15,-0.21,0.09,-0.04,-0.08,-0.13,-0.16


In [54]:
processed_data_stockel = new_df.reset_index()

In [56]:
processed_data_stockel.head()

,Contig,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9
0,Contig0.10_10_14370_15446,cce_4187,-0.74,-0.62,-0.13,0.35,0.26,-0.16,-0.87,-0.43,-0.12,0.38,0.37,-0.04
1,Contig0.10_11_16336_15422,cce_4186,-0.33,-0.64,-0.31,0.16,0.41,0.23,-0.42,-0.45,-0.30,0.06,0.34,0.23
2,Contig0.10_12_16503_17552,cce_4185,-0.46,-0.41,-0.41,0.66,-0.20,-0.04,-0.46,-0.28,-0.46,0.48,-0.01,0.04
3,Contig0.10_13_17679_17981,cce_4184,-0.23,0.27,0.13,0.08,-0.12,-0.08,-0.23,0.11,0.12,0.18,-0.05,-0.08
4,Contig0.10_14_18000_18698,cce_4183,-0.18,0.18,-0.02,-0.09,-0.22,-0.15,-0.21,0.09,-0.04,-0.08,-0.13,-0.16


*Will use this data for analysis*

In [63]:
processed_data_stockel.to_csv('MicroarrayData/StockelProcessed.csv',index=False)